# 🧪 Laboratoire: EDA → Nettoyage → Modélisation (propriété)
        
**Objectifs pédagogiques**
- Charger et explorer un jeu de données réel (`property.csv`).
- Évaluer la qualité des données (types, valeurs manquantes, doublons, outliers).
- Nettoyer et prétraiter (imputation, encodage, normalisation).
- Entraîner et comparer plusieurs modèles de régression.
- Interpréter les résultats et sauvegarder les artefacts.

**Contexte**
On suppose que l'on souhaite **prédire un prix** (ex.: `SalePrice`, `Price`, etc.). Si la colonne cible s’appelle différemment, ajustez `TARGET_COL` ci-dessous.


## 1) Imports & configuration

In [ ]:
# Imports standards
import os
import json
import pickle
from typing import List

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Pour l'affichage complet des colonnes
pd.set_option('display.max_columns', 100)

# Fichier source (déjà chargé sur la plateforme)
CSV_PATH = '/mnt/data/property.csv'

# Si vous connaissez déjà la cible, mettez-la ici. Sinon laissez None pour inférence automatique.
TARGET_COL = None  # Exemple: 'SalePrice' ou 'Price'

# Utiliser un style par défaut (ne PAS fixer de couleurs selon les consignes)
# plt.style.use('default')  # déjà par défaut


## 2) Chargement des données

In [ ]:
assert os.path.exists(CSV_PATH), f'Fichier introuvable: {CSV_PATH}'
df = pd.read_csv(CSV_PATH)
print(df.shape)
df.head()

## 3) Structure, types et valeurs manquantes

In [ ]:
print("Nombre de lignes:", len(df))
print("Nombre de colonnes:", df.shape[1])
print("\nTypes de données:")
print(df.dtypes)

missing = df.isna().sum().sort_values(ascending=False)
missing_pct = (df.isna().mean()*100).sort_values(ascending=False)
display(pd.DataFrame({'missing_count': missing, 'missing_pct': missing_pct}).head(20))

## 4) Définir la colonne cible (target)

In [ ]:
# Heuristique pour inférer la cible si non fournie
if TARGET_COL is None:
    candidates = [c for c in df.columns if 'price' in c.lower()] or df.select_dtypes(include=[np.number]).columns.tolist()[-1:]
    TARGET_COL = candidates[0] if candidates else None

print("Colonne cible candidate:", TARGET_COL)
assert TARGET_COL is not None, "Impossible d'inférer la colonne cible. Spécifiez TARGET_COL manuellement."

# Convertir la cible en numérique si nécessaire et drop les NA cible
df[TARGET_COL] = pd.to_numeric(df[TARGET_COL], errors='coerce')
df = df[~df[TARGET_COL].isna()].copy()
print("Taille après suppression des NA sur la cible:", df.shape)

## 5) Statistiques descriptives et visualisations

In [ ]:
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = [c for c in df.columns if c not in numeric_cols]

# Stats numériques
desc = df[numeric_cols].describe().T
desc

In [ ]:
# Distribution de la cible
plt.figure()
df[TARGET_COL].hist(bins=30)
plt.title(f"Distribution de la cible: {TARGET_COL}")
plt.xlabel(TARGET_COL)
plt.ylabel("Count")
plt.tight_layout()
plt.show()

In [ ]:
# Distributions de quelques variables numériques
for c in numeric_cols[:6]:
    plt.figure()
    df[c].hist(bins=30)
    plt.title(f"Distribution: {c}")
    plt.xlabel(c)
    plt.ylabel("Count")
    plt.tight_layout()
    plt.show()

In [ ]:
# Carte de corrélation (limiter à 20 colonnes pour la lisibilité)
corr_cols = numeric_cols[:20]
if len(corr_cols) >= 2:
    corr = df[corr_cols].corr(numeric_only=True)
    plt.figure()
    plt.imshow(corr, aspect='auto')
    plt.xticks(range(len(corr_cols)), corr_cols, rotation=90)
    plt.yticks(range(len(corr_cols)), corr_cols)
    plt.title("Heatmap des corrélations")
    plt.colorbar()
    plt.tight_layout()
    plt.show()

## 6) Qualité des données: doublons, outliers, imputation

In [ ]:
# a) Doublons
before = len(df)
df = df.drop_duplicates()
print("Doublons supprimés:", before - len(df))

# b) Winsorization IQR simple pour limiter l'impact des outliers
def winsorize_iqr(s, factor=1.5):
    q1 = s.quantile(0.25)
    q3 = s.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - factor*iqr
    upper = q3 + factor*iqr
    return s.clip(lower, upper)

for c in numeric_cols:
    if c == TARGET_COL:
        continue
    df[c] = winsorize_iqr(df[c])

# c) Imputation: num -> médiane ; cat -> mode
for c in numeric_cols:
    df[c] = df[c].fillna(df[c].median())

for c in categorical_cols:
    if df[c].isna().any():
        mode_val = df[c].mode(dropna=True)
        mode_val = mode_val.iloc[0] if not mode_val.empty else "Unknown"
        df[c] = df[c].fillna(mode_val)

print("Taille finale (post-nettoyage):", df.shape)

## 7) (Optionnel) Feature engineering

In [ ]:
# Exemple (adapter selon le dataset)
# if 'year_built' in df.columns and 'year_sold' in df.columns:
#     df['house_age_at_sale'] = df['year_sold'] - df['year_built']

# if 'lot_area' in df.columns and 'living_area' in df.columns:
#     df['area_ratio'] = df['living_area'] / (df['lot_area'] + 1e-6)

print("Colonnes disponibles après FE (si appliqué):", df.columns.tolist()[:20], "...")

## 8) Split & preprocessing (ColumnTransformer)

In [ ]:
feature_cols = [c for c in df.columns if c != TARGET_COL]
X = df[feature_cols].copy()
y = df[TARGET_COL].copy()

X_num = X.select_dtypes(include=[np.number]).columns.tolist()
X_cat = [c for c in X.columns if c not in X_num]

num_tf = Pipeline(steps=[("scaler", StandardScaler(with_mean=False))])
cat_tf = Pipeline(steps=[("onehot", OneHotEncoder(handle_unknown='ignore', sparse=True))])

preprocess = ColumnTransformer(
    transformers=[('num', num_tf, X_num), ('cat', cat_tf, X_cat)],
    remainder='drop',
    sparse_threshold=0.3
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
len(X_train), len(X_test)

## 9) Entraîner plusieurs modèles

In [ ]:
models = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(alpha=1.0, random_state=42),
    "Lasso": Lasso(alpha=0.001, max_iter=10000, random_state=42),
    "RandomForest": RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1),
    "GradientBoosting": GradientBoostingRegressor(random_state=42),
}

results = []
fitted = {}

for name, est in models.items():
    pipe = Pipeline(steps=[('preprocess', preprocess), ('model', est)])
    pipe.fit(X_train, y_train)
    fitted[name] = pipe
    y_pred = pipe.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r2 = r2_score(y_test, y_pred)
    results.append({'model': name, 'MAE': mae, 'RMSE': rmse, 'R2': r2})

pd.DataFrame(results).sort_values('RMSE')

## 10) Interprétation: importances / coefficients

In [ ]:
def feature_names_from_preprocessor(preproc: ColumnTransformer) -> List[str]:
    names = []
    for name, trans, cols in preproc.transformers_:
        if name == 'num':
            names.extend(list(cols))
        elif name == 'cat':
            ohe = trans.named_steps['onehot']
            names.extend(ohe.get_feature_names_out(cols).tolist())
    return names

best_name = pd.DataFrame(results).sort_values('RMSE').iloc[0]['model']
best_pipe = fitted[best_name]
print("Meilleur modèle:", best_name)

names = feature_names_from_preprocessor(best_pipe.named_steps['preprocess'])
model = best_pipe.named_steps['model']

importances_df = None
if hasattr(model, 'feature_importances_'):
    importances_df = pd.DataFrame({'feature': names, 'importance': model.feature_importances_})                         .sort_values('importance', ascending=False).head(25)
elif hasattr(model, 'coef_'):
    coef = model.coef_
    if hasattr(coef, 'toarray'):
        coef = coef.toarray().ravel()
    importances_df = pd.DataFrame({'feature': names, 'coefficient': coef})
    importances_df['abs_coef'] = importances_df['coefficient'].abs()
    importances_df = importances_df.sort_values('abs_coef', ascending=False).drop(columns=['abs_coef']).head(25)

importances_df

In [ ]:
# Visualisation (barres horizontales) — une figure par graphique, couleurs par défaut
if importances_df is not None:
    plt.figure()
    col = 'importance' if 'importance' in importances_df.columns else 'coefficient'
    vals = importances_df[col].values
    labels = importances_df['feature'].values
    plt.barh(range(len(vals)), vals)
    plt.yticks(range(len(vals)), labels)
    plt.title(f"Top features / coefficients — {best_name}")
    plt.tight_layout()
    plt.show()

## 11) Sauvegarder les artefacts (données, métriques, modèle)

In [ ]:
clean_path = '/mnt/data/property_cleaned_lab.csv'
metrics_path = '/mnt/data/model_metrics_lab.csv'
model_path = f'/mnt/data/best_model_{best_name}_lab.pkl'

# Sauvegarder données nettoyées
df.to_csv(clean_path, index=False)

# Sauvegarder métriques: reprendre results
metrics_df = pd.DataFrame(results).sort_values('RMSE')
metrics_df.to_csv(metrics_path, index=False)

# Sauvegarder le pipeline entraîné
with open(model_path, 'wb') as f:
    pickle.dump({'pipeline': best_pipe, 'target': TARGET_COL, 'feature_cols': X.columns.tolist()}, f)

print('Artifacts:')
print(' - Données nettoyées :', clean_path)
print(' - Métriques         :', metrics_path)
print(' - Modèle            :', model_path)

## 12) Questions de réflexion (à discuter en classe)
1. Quelles colonnes semblent le plus corrélées à la cible ? Pourquoi ?  
2. L'IQR capping (winsorization) a-t-il changé significativement les distributions ?  
3. Quel modèle performe le mieux et pourquoi pensez-vous que c'est le cas sur ce dataset ?  
4. Que feriez-vous pour améliorer la performance (features, hyperparamètres, CV) ?  
5. Si vous déployiez ce modèle, quelles considérations de **biais/éthique** ou de **vie privée** seraient pertinentes ?